# requireJS -- load a define/require style javascript module

We want to load the following module:

In [ ]:
import jp_proxy_widget
from jp_proxy_widget import js_context

In [ ]:
require_fn="js/simple_define.js"
print(js_context.get_text_from_file_name(require_fn))

In [ ]:
requireJS = jp_proxy_widget.JSProxyWidget()

In [ ]:
# callback for storing the styled element color
requireJSinfo = {}

def require_answer_callback(answer):
    requireJSinfo["answer"] = answer
    
module_identifier = "three_primes"

requireJS.require_js(module_identifier, require_fn)

# initialize the element using code that requires the loaded module
requireJS.js_init("""

console.log("js init calling requirejs " + module_identifier);

element.html("<em>loading " + module_identifier + "</em>")

element.requirejs([module_identifier], function(module_value) {
    console.log("js init using value for " + module_identifier);

    element.html('<b>First three primes: ' + module_value.first_3_primes + '</b>')

    require_answer_callback(module_value);
});

""", require_answer_callback=require_answer_callback, module_identifier=module_identifier)
    
#requireJS.uses_require(after_load)
#after_load()
                      
def validate_requireJS():
    expect = {"first_3_primes": [2,3,5]}
    assert expect == requireJSinfo["answer"], repr((expect, requireJSinfo))
    assert requireJS._require_checked
    print ("Loaded requirejs value is correct!")

In [ ]:
#requireJS.commands_awaiting_render

In [ ]:
requireJS

In [ ]:
import notebook_test_helpers

validators = notebook_test_helpers.ValidationSuite()
validators.add_validation(requireJS, validate_requireJS)

# A more realistic example `FileSaver.js`

# WARNING: Running the widget will cause the browser to download a small text file to your downloads area


In [ ]:
class SaverWidget(jp_proxy_widget.JSProxyWidget):
    
    def __init__(self, *pargs, **kwargs):
        super(SaverWidget, self).__init__(*pargs, **kwargs)
        # Wiring: set up javascript callables and python callbacks.
        self.require_js("saveAs", "js/FileSaver.js")
        self.js_init("""
        debugger;
        element.html("Requiring saveAs...");

        element.requirejs(["saveAs"], function(saveAs) {
            element.html("saveAs = " + saveAs);
            element.download = function(text, name, type) {
                if (!type) {
                    type="text/plain;charset=utf-8";
                }
                var blob = new Blob([text], {type: type});
                element.html("Now saving " + text.length + " as " + name + " with type " + type);
                saveAs(blob, name);
                confirm(text, name, type);
            };
            ready();
        });
        """, confirm=self.confirm, ready=self.ready)
    
    is_ready = False
        
    def ready(self):
        "call this when the widget has fully initialized.  Download a very small text file."
        self.is_ready = True
        saverWidget.element.download("Not very interesting text file content.", "Save_as_test.txt")

    confirmed = None
    
    def confirm(self, *args):
        self.confirmed = args

saverWidget = SaverWidget()

def ready():
    "call this when the widget hsa fully initialized.  Download a very small text file."
    saverWidget.element.download("Not very interesting text file content.", "Save_as_test.txt")

saverWidget

In [ ]:
expected_confirmation = [
 'Not very interesting text file content.',
 'Save_as_test.txt',
 'text/plain;charset=utf-8']

def validateSaver():
    assert saverWidget.is_ready
    assert saverWidget.confirmed is not None, "confirmation hasn't happened yet."
    assert list(saverWidget.confirmed) == list(expected_confirmation)
    print ("file saver apparently worked!")

validators.add_validation(saverWidget, validateSaver)

In [ ]:
validators.run_all_in_widget()